In [67]:
import time
import os
import random
import logging
import pandas as pd
from openpyxl import load_workbook

In [68]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [69]:
# Ciblage du dossier parent
dossier_parent = os.path.dirname(os.getcwd())

# Chemin absolu du fichier .csv
dico = dossier_parent + "\\dico.csv"

In [70]:
# Initialisation du dataframe Pandas
df = pd.read_csv(dico)
df = df.sort_values("Mot")
df = df.dropna()
df = df.reset_index(drop=True)

In [71]:
# Ouverture du fichier excel
excel = load_workbook(filename="lexi.xlsx")
sheet = excel.active

In [72]:
sheet["A1"] = "Mot"
sheet["B1"] = "Définitions"
sheet["C1"] = "Timestamp"

In [73]:
def first_empty():
    """ Ruturns the index of the first empty cell
    in column A """
    idx = 2
    while True:
        if not isinstance(sheet[f'A{idx}'].value, str):
            return idx
        else:
            idx+=1
            continue

In [74]:
def ajout(mot, print_added=False):
    
    mot = mot.capitalize()

    D = df.loc[df['Mot'] == mot]['Définitions']

    # Mot introuvable
    if len(D) == 0:
        logging.info(f"'{mot}' not in dictionnary")
        #print(f"'{mot}' ne figure pas dans le dictionnaire")
        return None

    # Occurence trouvée
    else:
        logging.info(f"'{mot}' found in dictionnary at idx {df.loc[df['Mot'] == mot].index[0]}")
        D = eval(D.values[0]) # list

        idx = 2 # Index vertical de la colonne

        # Boucle à la recherche d'une case vide
        while idx < first_empty():

            # Le mot est déjà présent
            if sheet[f"A{idx}"].value == mot:
                logging.warning(f"'{mot}' not added : already in lexicon at cell A{idx}")
                #print(f"Le mot '{mot}' existe déjà à la cellule A{idx}")
                return None

            # La cellule ne contient pas de texte
            if not isinstance(sheet[f"A{idx}"].value, (str, int, float)) :

                # Toutes les définitions du mot contenues dans
                # la liste (D) sont fusionnées dans une seule 
                # chaîne de caractère afin d'être insérée dans 
                # la cellule du tableur
                defs = []
                for idx_def, definition in enumerate(D):
                    defs.append(f"{idx_def+1}) {definition}")

                sheet[f"A{str(idx)}"] = mot # Ajout mot

                sheet[f"B{str(idx)}"] = "".join(defs) # Ajout définition
                
                sheet[f"C{str(idx)}"] = str(int(time.time())) # Ajout timestamp

                logging.info(f"'{mot}' added in lexicon at idx A{idx}")
                if print_added:
                    print(f"Définition de '{mot}' ajoutée")

                break
            
            else:
                idx+=1
                if idx == 10000: # watchdog
                    logging.critical("Watchdog activated : too many iterations")
                    break
    
    excel.save(filename="lexi.xlsx")
    logging.debug("xlsx file saved after adding a word")

In [75]:
def search(mot):
    """ Search in lexicon """
    mot = mot.capitalize()

    idx = 2
    found = False
    while idx < first_empty():
        if sheet[f'A{idx}'].value == mot:
            found = True
            logging.info(f"'{mot}' found in lexicon at idx {idx}")
            return (idx, mot, sheet[f'B{idx}'].value)
        else:
            idx+=1
    
    if found == False:
        logging.info(f"'{mot}' not found in lexicon")
        #print(f"'{mot}' not found")
        return None

In [76]:
def delete(mot):
    """ Delete a word from the lexicon """
    mot = mot.capitalize()

    result = search(mot)

    if result == None:
        logging.warning(f"Word '{mot}' not deleted : not in lexicon")
        return None
    
    else:
        idx = result[0]
        sheet.delete_rows(idx)
        logging.info(f"'{mot}' deleted from the lexicon")
        #print(f"Word '{mot}' deleted at row {idx}")
        excel.save("lexi.xlsx")
        logging.debug("xlsx file saved after deletion")


In [77]:
def insert(mot, definition):
    """ Insert a word in lexicon that not figure in dictionary """
    mot = mot.capitalize()

    result = search(mot)

    if result != None:
        idx = result[0]
        logging.warning(f"Word '{mot}' not inserted : already in lexicon at idx {idx}")
        #print(f"Word '{mot}' already present in row {idx}")
        return None
    
    else:
        row = first_empty()

        definition_txt = ""

        if isinstance(definition, (list, tuple)):
            for idx, d in enumerate(definition):
                definition_txt += f"{idx+1}) {d} "
                sheet[f'A{row}'] = mot
                sheet[f'B{row}'] = definition_txt

        elif isinstance(definition, (str)):
            definition_txt = "1) " + definition
            sheet[f'A{row}'] = mot
            sheet[f'B{row}'] = definition_txt
            #sheet["A1"] = "Mot"

        else:
            logging.critical(f"Word '' not inserted : definition must be str or strs in list/tuple")
            return None
        
        excel.save("lexi.xlsx")
        logging.debug("xlsx file saved after insertion")

In [78]:
def add_random_words(sample_lenght):
    sample = df.sample(sample_lenght)

    for idx, mot in enumerate(sample["Mot"]):
        ajout(mot)
    
    excel.save("lexi.xlsx")
    logging.debug("xlsx file saved after adding random sample")

In [79]:
#add_random_words(100)

In [80]:
insert("test_word", ["test_word_def_1", "test_word_def_2"])

INFO:root:'Test_word' found in lexicon at idx 100


In [85]:
delete("test_word")

INFO:root:'Test_word' found in lexicon at idx 100
INFO:root:'Test_word' deleted from the lexicon


In [81]:
first_empty()

101

In [82]:
ajout("bouger", print_added=True)

INFO:root:'Bouger' found in dictionnary at idx 90518


In [83]:
# Parcours d'un intervalle
col_range_gen = sheet.iter_cols(min_col=1, max_col=1, min_row=2, max_row=10)

T = ()
for col_tuple in col_range_gen:
    T = col_tuple

for cell in T:
    print(cell.value)

Farandole
Cavaleras
Rouillures
Saint-pierrards
Nourrisseur
Alingèrent
Faséieront
Génuflexionnai
Rejaugerons
